In [14]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.models import vgg16
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt
from modeling.train_utils import train_loop_with_scheduler

In [2]:
PS_labels = np.load('../data/Processed_pattern_stimuli/labels.npy')
# new_PS_labels = np.zeros((PS_labels.size, PS_labels.max() + 1))
# new_PS_labels[np.arange(PS_labels.size), PS_labels] = 1
PS_labels_all = np.concatenate((PS_labels, PS_labels, PS_labels, PS_labels), 0)

In [4]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
PS_set1 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_1.npy')
PS_set2 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_2.npy')
PS_set3 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_3.npy')
PS_set4 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_4.npy')
PS_imgs_all = np.concatenate((PS_set1, PS_set2, PS_set3, PS_set4), 0)
PS_imgs_all = np.reshape(PS_imgs_all, (PS_imgs_all.shape[0],1,50, 50))

#tensor_imgs = torch.zeros((PS_imgs_all.shape[0], 1, 50, 50))

PS_imgs_all = torch.tensor(PS_imgs_all)
# for i, x in enumerate(PS_imgs_all):
#     img = Image.fromarray(x).convert('RGB')
#     input_tensor = preprocess(img)
#     tensor_imgs[i] = input_tensor

In [3]:
torch.save(tensor_imgs, '../data/Processed_pattern_stimuli/all_imgs_tensor.pt')

In [6]:
PS_imgs_all = torch.load('../data/Processed_pattern_stimuli/all_imgs_tensor.pt')

In [4]:

class Vgg16(torch.nn.Module):
    def __init__(self):
        super(Vgg16, self).__init__()
        features = list(vgg16(pretrained=True).features)
        # features的第3，8，15，22层分别是: relu1_2,relu2_2,relu3_3,relu4_3
        self.features = torch.nn.ModuleList(features).eval()

    def forward(self, x):
        results = []
        for ii, model in enumerate(self.features):
            x = model(x)
            if ii in {23}:
                results.append(x)
        return results


class label_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(16 * 512, 2048)
        self.linear2 = nn.Linear(2048, 6)

    def forward(self, x):
        output = self.linear1(x)
        output = self.linear2(output)
        return output
class label_model_CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=30, kernel_size=(5, 5), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
            nn.Conv2d(in_channels=30, out_channels=30, kernel_size=(5, 5), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
            nn.Conv2d(in_channels=30, out_channels=30, kernel_size=(3, 3), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
            nn.Conv2d(in_channels=30, out_channels=30, kernel_size=(3, 3), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
        )
        self.flatten = nn.Flatten()
        self.Linear = nn.Linear(43320, 1)

    def forward(self, x):
        x = self.layers(x)
        x = self.flatten(x)
        x = self.Linear(x)
        return x

In [8]:
num_epochs = 15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Vgg_model = Vgg16()
# model = label_model()
# Vgg_model = Vgg_model.to(device)
# model = model.to(device)
# Vgg_model.eval()
model = label_model_CNN()
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_loss = 100
for epoch in range(num_epochs):
    total_loss = 0
    for x, y in tqdm(loader):
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)
        y = y.long()
        # tmp = Vgg_model(x)[0][:, :, 5:9, 5:9]
        # tmp = torch.reshape(tmp, (batchsize, 16*512))
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    total_loss /= len(loader)
    print(total_loss)
    if total_loss < best_loss:
        torch.save(model.state_dict(),'A:/school/College_Junior/research/CNN_Tang_project/saved_models/PS_labeling_model_CNN')

KeyboardInterrupt: 

In [12]:
batchsize = 100
p = torch.randperm(PS_labels_all.shape[0])

PS_labels_all = PS_labels_all[p]
PS_imgs_all = PS_imgs_all[p]

train_set = PS_imgs_all[:37000]
test_set = PS_imgs_all[37000:]

train_labels = PS_labels_all[:37000]
test_labels = PS_labels_all[37000:]

dataset = TensorDataset(train_set, torch.tensor(train_labels, dtype=torch.long))
loader = DataLoader(dataset, batch_size=batchsize, shuffle=True)

test_dataset = TensorDataset(test_set, torch.tensor(test_labels, dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=batchsize)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = label_model_CNN()
model = model.to(device)
num_epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network, losses, accs = train_loop_with_scheduler(loader, test_loader, model, optimizer,
        criterion, criterion, num_epochs, device=device,
        save_location = 'A:/school/College_Junior/research/CNN_Tang_project/saved_models/PS_labeling_model_CNN')

  0%|          | 0/15 [00:00<?, ?it/s]


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward

In [29]:
all_predicts = torch.zeros(299)
for neuron in range(299):
    vis_img = plt.imread("../analysis/Visualization/SCNN/visualization_seperate_cnn/" + str(neuron) + '.jpg')
    img = Image.fromarray(vis_img).convert('RGB')
    input_tensor = preprocess(img)
    input_tensor = torch.reshape(input_tensor, (1,3,224,224))
    x = input_tensor.to(device)
    tmp = Vgg_model(x)[0][:, :, 5:9, 5:9]
    tmp = torch.reshape(tmp, (1, 16*512))
    out = model(tmp)
    predict = torch.argmax(out)
    all_predicts[neuron] = predict

In [30]:
all_predicts = all_predicts.detach().cpu().numpy()

In [31]:
all_predicts = all_predicts.astype(int)
label_names = ['SS', 'EB', 'CN', 'CV', 'CRS', 'Other']
all_labels = []
for i, p in enumerate(all_predicts):
    all_labels.append(label_names[p])
